Refert to https://github.com/xiaoweiChen/Cpp_Concurrency_In_Action/blob/master/content/chapter4/4.4-chinese.md

## 4.4 使用同步操作简化代码

当需要为程序的并发时，这是一种可以帮助你简化你的代码的方式，提供更多的函数化的方法。**比起在多个线程间直接共享数据，每个任务拥有自己的数据会应该会更好**，并且结果可以对其他线程进行广播，这就需要使用“期望”来完成了。

### 4.4.1 使用“期望”的函数化编程
术语函数化编程(functional programming)引用于一种编程方式，这种方式中的函数结果**只依赖于传入函数的参数，并不依赖外部状态**。当一个函数与数学概念相关时，当你使用相同的函数调用这个函数两次，这两次的结果会完全相同。 

很容易想象这是一种什么样的情况，特别是当并行发生时，
- 因为在第三章时我们讨论过，很多问题发生在共享数据上。
- 当共享数据没有被修改，那么就不存在条件竞争，并且没有必要使用互斥量去保护共享数据。这可对编程进行极大的简化，例如Haskell语言[2]，在Haskell中函数默认就是这么的“纯粹”；这种纯粹对的方式，在并发编程系统中越来越受欢迎。
- 因为大多数函数都是纯粹的，那么非纯粹的函数对共享数据的修改就显得更为突出，所以其很容易适应应用的整体结构。

函数化编程的好处，并不限于那些将“纯粹”作为默认方式(范型)的语言。 **C++ 是一个多范型的语言，其也可以写出FP类型的程序**。
- 在 C++11 中这种方式要比 C++98 简单许多，因为 C++11 支持lambda表达式(详见附录A，A.6节)，还加入了Boost和TR1中的 std::bind ，以及自动可以自行推断类型的自动变量(详见附录A，A.7节)。
- “期望”作为拼图的最后一块，它使得函数化编程模式并发化(FP-style concurrency)在 C++ 中成为可能；
- **一个“期望”对象可以在线程间互相传递，并允许其中一个计算结果依赖于另外一个的结果，而非对共享数据的显式访问。**

#### 清单4.12 快速排序——顺序实现版

In [1]:
#include <iostream>
#include <list>
#include <algorithm>

namespace n1 {
    template<typename T>
    std::list<T> sequential_quick_sort(std::list<T> input)
    {
        if (input.empty()) {
            return input;
        }

        std::list<T> result;
        result.splice(result.begin(), input, input.begin()); // 1
        T const& pivot = *result.begin(); // 2
        auto divide_point = std::partition(input.begin(), input.end(), [&](T const& t) {
            return t < pivot;
        }); // 3

        std::list<T> lower_part;
        lower_part.splice(lower_part.end(), input, input.begin(), divide_point); // 4
        auto new_lower(sequential_quick_sort(std::move(lower_part))); // 5
        auto new_higher(sequential_quick_sort(std::move(input))); // 6

        result.splice(result.end(), new_higher); // 7
        result.splice(result.begin(), new_lower); // 8
        return result;
    }
} // namespace n1

In [2]:
{
    std::list<int> l1{ 2, 3, 39, -1, 234, 3 };
    auto l2 = n1::sequential_quick_sort(l1);
    for (auto i : l2) {
        std::cout << i << ' ';
    }
}

-1 2 3 3 39 234 

虽然**接口的形式是FP模式的，但当你使用FP模式时，你需要做大量的拷贝操作**，所以在内部你会使用“普通”的命令模式。
1. 你选择第一个数为“中间”值，**使用splice()①将输入的首个元素(中间值)放入结果列表中**。虽然这种方式产生的结果可能不是最优的(会有大量的比较和交换操作)，但是对 std::list 做任何事都需要花费较长的时间，因为链表是遍历访问的。你知道你想要什么样的结果，所以你可以直接将要使用的“中间”值提前进行拼接。
- 现在你还需要使用“中间”值进行比较，所以这里使用了一个引用②，为了避免过多的拷贝。
- 之后，你可以使用 std::partition 将序列中的值分成小于“中间”值的组和大于“中间”值的组③。最简单的方法就是使用lambda函数指定区分的标准；使用已获取的引用避免对“中间”值的拷贝. std::partition() 对列表进行重置，并返回一个指向首元素(不小于“中间”值)的迭代器。迭代器的类型全称可能会很长，所以你可以使用auto类型说明符，让编译器帮助你定义迭代器类型的变量
- 现在，你已经选择了FP模式的接口；所以，当你要使用递归对两部分排序是，你将需要创建两个列表。你可以用splice()函数来做这件事，将input列表小于divided_point的值移动到新列表lower_part④中。
- 其他数继续留在input列表中。而后，你可以使用递归调用⑤⑥的方式，对两个列表进行排序。
- 这里**显式使用 std::move() 将列表传递到类函数中，这种方式还是为了避免大量的拷贝操作。**
- 最终，你可以再次使用splice()，将result中的结果以正确的顺序进行拼接。new_higher指向的值放在“中间”值的后面⑦，
- new_lower指向的值放在“中间”值的前面⑧。

**NOTE:** 该实现的亮点：
- FP的接口，但内部实现仍然利用CPP语言特性兼顾效率
- list::splice 和 std::move 避免了拷贝和内存重新分配

#### 清单4.13 快速排序——“期望”并行版


In [3]:
#include <iostream>
#include <list>
#include <algorithm>
#include <future>

namespace n2 {
    template<typename T>
    std::list<T> parallel_quick_sort(std::list<T> input)
    {
        if (input.empty()) {
            return input;
        }

        std::list<T> result;
        result.splice(result.begin(), input, input.begin());
        T const& pivot = *result.begin();
        auto divide_point = std::partition(input.begin(), input.end(),
            [&](T const& t) {return t < pivot; });

        std::list<T> lower_part;
        lower_part.splice(lower_part.end(), input, input.begin(), divide_point);
        std::future<std::list<T> > new_lower( // 1
            std::async(&parallel_quick_sort<T>, std::move(lower_part)));
        auto new_higher(parallel_quick_sort(std::move(input))); // 2

        result.splice(result.end(), new_higher); // 3
        result.splice(result.begin(), new_lower.get()); // 4
        return result;
    }
} // namespace n2

In [4]:
{
    std::list<int> l1{ 2, 3, 39, -1, 234, 3 };
    auto l2 = n2::parallel_quick_sort(l1);
    for (auto i : l2) {
        std::cout << i << ' ';
    }
}
/* Output:
-1 2 3 3 39 234 
*/

IncrementalExecutor::executeFunction: symbol '__emutls_v._ZSt11__once_call' unresolved while linking [cling interface function]!
IncrementalExecutor::executeFunction: symbol '__emutls_v._ZSt15__once_callable' unresolved while linking [cling interface function]!


这里最大的变化是，
1. 当前线程不对小于“中间”值部分的列表进行排序，使用 std::async() ①在另一线程对其进行排序。
- 大于部分列表，如同之前一样，使用递归的方式进行排序②。通过递归调用parallel_quick_sort()，你就可以利用可用的硬件并发了。

std::async() 会启动一个新线程，这样当你递归三次时，就会有八个线程在运行了；当你递归十次(对于大约有1000个元素的列表)，如果硬件能处理这十次递归调用，你将会创建1024个执行线程。**当运行库认为这样做产生了太多的任务时(也许是因为数量超过了硬件并发的最大值)，运行库可能会同步的切换新产生的任务。**当任务过多时(已影响性能)，这些任务应该在使用get()函数获取的线程上运行，而不是在新线程上运行，这样就能避免任务向线程传递的开销。值的注意的是，这完全符合 std::async 的实现，为每一个任务启动一个线程(甚至在任务超额时；在 std::launch::deferred 没有明确规定的情况下)；或为了同步执行所有任务(在 std::launch::async 有明确规定的情况下)。当你依赖运行库的自动缩放，建议你去查看一下你的实现文档，了解一下将会有怎么样的行为表现。

比起使用 std::async() ，你可以写一个spawn_task()函数对 std::packaged_task 和 std::thread 做简单的包装，如清单4.14中的代码所示；你需要为函数结果创建一个 std::packaged_task 对象， 可以从这个对象中获取“期望”，或在线程中执行它，返回“期望”。**其本身并不提供太多的好处(并且事实上会造成大规模的超额任务)，**但是它会为转型成一个更复杂的实现铺平道路，将会实现向一个队列添加任务，**而后使用线程池的方式来运行它们**。我们将在第9章再讨论线程池。**使用 std::async 更适合于当你知道你在干什么，并且要完全控制在线程池中构建或执行过任务的线程。**

#### 清单4.14 spawn_task的简单实现


In [5]:
#include <future>

namespace n3 {
    template<typename F, typename A>
    std::future<typename std::result_of<F(A&&)>::type>
        spawn_task(F&& f, A&& a)
    {
        typedef typename std::result_of<F(A&&)>::type result_type;
        std::packaged_task<result_type(A&&)> task(std::move(f));
        std::future<result_type> res(task.get_future());
        std::thread t(std::move(task), std::move(a));
        t.detach();
        return res;
    }
} // namespace n3

其他先不管，回到parallel_quick_sort函数。
- 因为你只是直接递归去获取new_higher列表，你可以如之前一样对new_higher进行拼接③。
- 但是，new_lower列表是 `std::future<std::list<T>>`的实例，而非是一个简单的列表，所以你需要调用get()成员函数在调用splice()④之前去检索数值。
- 在这之后，等待后台任务完成，并且将结果移入splice()调用中；
- **get()返回一个包含结果的右值引用，所以这个结果是可以移出的**

即使假设，使用 std::async() 是对可用硬件并发最好的选择，但是**这样的并行实现对于快速排序来说，依然不是最理想的**。其中， std::partition 做了很多工作，即使做了依旧是顺序调用，但就现在的情况来说，已经足够好了。如果你对实现最快并行的可能性感兴趣的话，你可以去查阅一些学术文献。

因为避开了共享易变数据，函数化编程可算作是并发编程的范型；并且也是**通讯顺序进程(CSP,Communicating Sequential Processer[3],)的范型**，这里线程理论上是完全分开的，也就是没有共享数据，但是有通讯通道允许信息在不同线程间进行传递。这种范型被Erlang语言所采纳，并且在MPI(Message Passing Interface，消息传递接口)上常用来做C和 C++ 的高性能运算。


### 4.4.2 使用消息传递的同步操作

CSP的概念十分简单：
- 当没有共享数据，每个线程就可以进行独立思考，其行为纯粹基于其所接收到的信息。
- 每个线程就都有一个状态机：当线程收到一条信息，它将会以某种方式更新其状态，并且可能向其他线程发出一条或多条信息，对于消息的处理依赖于线程的初始化状态。
- 这是一种正式写入这些线程的方式，**并且以有限状态机的模式实现，但是这不是唯一的方案；状态机可以在应用程序中隐式实现。**
- 这种方法在任何给定的情况下，都更加**依赖于特定情形下明确的行为要求和编程团队的专业知识。**
- 无论你选择用什么方式去实现每个线程，**任务都会分成独立的处理部分，这样会消除潜在的混乱(数据共享并发)**，这样就让编程变的更加简单，且拥有低错误率。

真正通讯顺序处理是没有共享数据的，所有消息都是通过消息队列传递，**但是因为 C++ 线程共享一块地址空间，所以达不到真正通讯顺序处理的要求**。这里就需要有一些约定了：
- 作为一款应用或者是一个库的作者，我们有责任**确保在我们的实现中，线程不存在共享数据。**
- 当然，为了线程间的通信，**消息队列是必须要共享的**，具体的细节可以包含在库中。


试想，有一天你要为实现ATM(自动取款机)写一段代码。这段代码需要处理，
- 人们尝试取钱时和银行之间的交互情况，
- 以及控制物理器械接受用户的卡片，显示适当的信息，处理按钮事件，吐出现金，还有退还用户的卡。

一种处理所有事情的方法是让代码将所有事情分配到三个独立线程上去：
- 一个线程去处理物理机械，
- 一个去处理ATM机的逻辑，
- 还有一个用来与银行通讯。

**这些线程可以通过信息进行纯粹的通讯，而非共享任何数据**。比如，当有人在ATM机上插入了卡片或者按下按钮，处理物理机械的线程将会发送一条信息到逻辑线程上，并且逻辑线程将会发送一条消息到机械线程，告诉机械线程可以分配多少钱，等等。

**一种为ATM机逻辑建模的方式是将其当做一个状态机。**线程的每一个状态都会等待一条可接受的信息，这条信息包含需要处理的内容。这可能会让线程过渡到一个新的状态，并且循环继续。

![](https://github.com/xiaoweiChen/Cpp_Concurrency_In_Action/blob/master/images/chapter4/4-3.png?raw=true)

完成示例：https://github.com/xiaoweiChen/Cpp_Concurrency_In_Action/blob/master/content/appendix_C/C.0-chinese.md